In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

import os
from distutils.dir_util import copy_tree, remove_tree

from PIL import Image
from random import randint

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow_addons as tfa
from keras.utils.vis_utils import plot_model
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D


print("TensorFlow Version:", tf.__version__)

C:\Users\KIIT\anaconda_works\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


TensorFlow Version: 2.12.0


In [3]:
images_ds = tf.data.Dataset.list_files('1/*/*/*', shuffle=True)

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: 1/*/*/*'

In [ ]:
image_count = len(images_ds)
image_count

In [ ]:
for file in images_ds.take(8):
    print(file.numpy())

In [ ]:
images_ds = images_ds.shuffle(6400)
for file in images_ds.take(400):
    print(file.numpy())

In [ ]:

class_names = ["MildDemented","VeryMildDemented","ModerateDemented","NonDemented"]

In [7]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [8]:
len(train_ds)

0

In [9]:
len(test_ds)

0

In [10]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [11]:
get_label('Alzheimer_s Dataset\\train\\VeryMildDemented\\verymildDem1159.jpg')

<tf.Tensor: shape=(), dtype=string, numpy=b'VeryMildDemented'>

In [12]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [32,32])
    return img, label

In [13]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)



ValueError: in user code:

    File "C:\Users\KIIT\AppData\Local\Temp\ipykernel_21644\2573562160.py", line 2, in process_image  *
        label = get_label(file_path)
    File "C:\Users\KIIT\AppData\Local\Temp\ipykernel_21644\326817248.py", line 3, in get_label  *
        parts = tf.strings.split(file_path, os.path.sep)

    ValueError: input: Tensor conversion requested dtype string for Tensor with dtype float32: <tf.Tensor 'args_0:0' shape=(None, 224, 224, None) dtype=float32>


In [ ]:
for image, label in train_ds.take(3):
    print("****",image)
    print("****",label)

In [ ]:
def scale(image, label):
    return image/255, label

In [ ]:

train_ds = train_ds.map(scale)

In [ ]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

In [ ]:
#WORK_DIR=
ZOOM = [.99, 1.01]
BRIGHT_RANGE = [0.8, 1.2]
HORZ_FLIP = True
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"

work_dr = IDG(rescale = 1./255, brightness_range=BRIGHT_RANGE, zoom_range=ZOOM, data_format=DATA_FORMAT, fill_mode=FILL_MODE, horizontal_flip=HORZ_FLIP)

train_data_gen = work_dr.flow_from_directory(directory=WORK_DIR, target_size=DIM, batch_size=6500, shuffle=False)

In [ ]:
show_images_with_labels(train_ds)